In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from DataAnalyzer import DataAnalyzer
spark = SparkSession.builder.appName("DataAnalyzing").getOrCreate()

In [ ]:
my_analyzer = DataAnalyzer(spark)
my_analyzer.load_data()

In [ ]:
my_analyzer.analyze()

Visualizations :

Fraud Suspected : 

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
sns.heatmap(my_analyzer.fraud_df.pivot('window_day', 'window_hour', 'number_of_customers'), annot=True, fmt='d', cmap='YlGnBu')
plt.title('Number of Customers suspected of fraud per hour')
plt.xlabel('Hour')
plt.ylabel('Day')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

Client per Segment:

In [ ]:
plt.figure(figsize=(8, 8))
plt.pie(my_analyzer.segment_df['count'], labels=my_analyzer.segment_df['segment'], autopct='%1.1f%%', startangle=140)
plt.title('Count per Group Segment')
plt.show()

Orders per cities: 

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='city', y='count', data=my_analyzer.cities_per_orders, palette='viridis')
plt.title('Customer Count per City')
plt.xlabel('City')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

Amount per countries:

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='country', y='sum(totalAmountUSD)', data=my_analyzer.countries_per_amount, palette='viridis')
plt.title('Total Amount per Country')
plt.xlabel('Country')
plt.ylabel('Total Amount')
plt.show()

Amounts per payment method:

In [ ]:
colors = sns.color_palette('viridis', len(my_analyzer.amount_per_payment_method))


plt.figure(figsize=(10, 8))
plt.pie(my_analyzer.amount_per_payment_method['sum(totalAmountUSD)'], labels=my_analyzer.amount_per_payment_method['paymentMethod'], autopct='%1.1f%%', startangle=140, colors=colors)
plt.title('Total Amount per Payment Method')
plt.show()